In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
from strategy_v2.Strategy import *
from strategy_v2.Strategy.MVO import *
from strategy_v2.Strategy.MVO.AlphaModel import *
from strategy_v2.Strategy.MVO.RiskModel import *
from strategy_v2.TradingSubSystem import *
from strategy_v2.Portfolio import *
from strategy_v2.TransactionCost import *
from strategy_v2.Executor import *
from utils.data_helper import *
from utils.data import *
from utils.performance import *
from utils.ta import *

In [2]:
instruments = [
    'META',
    'TSLA',
    'NVDA',
    'AAPL',    
    'DXJ',
    'BRK-B',
    'SPY',
    'QQQ',
    'NANC',
    #'BTC',    
    'CASH'
]

end_date = get_today(1)
start_date = pd.to_datetime(datetime(2024, 1, 3))
start_date = pd.to_datetime(datetime(2023, 2, 7))
max_leverage = 1
#vol_target = 0.25
vol_target = None
end_date

Timestamp('2025-01-21 00:00:00')

# Notes

- 2024-08-28: Tested expected return prediction with RandomForest, XGB, LGBM. None of them outperforms the SMA Model. LGBM is able to achieve a similar performance as SMA model and generally train faster.</br>

    | Measure                | ^SPX      | MVO - SMA1 | MVO - RandomForest1 | MVO - XGB1 | MVO - LGBM1 |
    |------------------------|-----------|------------|---------------------|------------|-------------|
    | Cumulative Return      | 1.173503  | 2.072994   | 1.662573            | 1.471717   | 2.023378    |
    | Annualized Return      | 0.076775  | 0.301537   | 0.221185            | 0.176047   | 0.296252    |
    | Annualized Volatility  | 0.180081  | 0.226417   | 0.240133            | 0.243953   | 0.243120    |
    | Annualized Sharpe Ratio| 0.240438  | 1.183923   | 0.781685            | 0.584416   | 1.080844    |
    | Maximum Drawdown       | -0.254251 | -0.271732  | -0.323658           | -0.380737  | -0.296535   |

    Models are shared the same hyperparameters: lookback (train days) = 10 days and gamma=10, hhi=0.2


## Leverage 
##### 1. capital (Stock MV / last leverage), 2. new leverage
2024-09-06: $169,336, 96.69%

2024-09-20: $172,168, 93.52%

2024-10-04: $176,235, 100%

2024-10-18: $177,831, 100%

2024-11-01: $175,558, 100%

2024-11-09: $181,511

2024-11-22: $187,224

2024-11-30: $188,332

2024-12-06: $194,828

2024-12-13: $211,354, CASH: $3778

2024-12-20: $198,713, CASH: $38,024 (+$20000 Capital)

2025-01-04: $192,246, CASH: $23,606

2024-01-10: $205,516, CASH: $18,562

2025-01-17: $213,405, CASH: $18,835

# Log

2024-11-22: If we don't fix sum of weights to constant (e.g. 1), the model would statisfy the HHI constraints by reducing the overeall weights only, but not diversing the portfolios. Still, fixing to 1 is not good because sometime we should de-leverage given signals are weak. So we can consider to add a cash element in the optimization

2024-11-30: Create benchmark portfolios (e.g. equally weighted of all stocks) (DONE)

2024-12-06: Alpha models using returns relative to something (e.g. SPX)

In [10]:
portfolio = PortfolioStandard(
    capital=213405+18562,
    name='MVOPortfolio',        
    rebalance_iter=RebalancerIter('0 0 * * Fri', 1),        
    tc_model=TransactionCostFutu(), 
    systems_style=SystemStyle.VERTICAL,
    systems=[        
        # Long terms signals => 60days return / diversified        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(60), risk_model=RollingMeanCovNeg(60), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Mid terms signals => 30days return / diversified        
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(30), risk_model=RollingMeanCovNeg(30), gamma=10, hhi=0.2, confidence=1, leverage=1)], max_leverage=max_leverage, offset=60),

        # Short terms signals => 5 and 10days returns / less diversified
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(10), risk_model=RollingMeanCovNeg(10), gamma=30, hhi=0.4, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RollingMean(5), risk_model=RollingMeanCovNeg(5), gamma=30, hhi=0.4, confidence=0.5, leverage=1)], max_leverage=max_leverage, offset=60),         

        # Mean Revert signals => 2days RSI / concentrated
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,10), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.2)], max_leverage=max_leverage, offset=200),
        # TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=RSI(2,95), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=-1)], max_leverage=max_leverage, offset=200),

        # Mean Revert signals => Double7 / concentrated
        TradingSubSystemSingle(vol_target=vol_target, instruments=instruments, strategy=[MeanVarianceOpt(alpha_model=Double7(5), risk_model=ZeroCov(), gamma=0, hhi=0, confidence=0.2)], max_leverage=max_leverage, offset=200),
    ]
)

portfolio.set_start_date(start_date)
portfolio.set_end_date(end_date)
portfolio.backtest_subsystems()
portfolio.optimize()
portfolio.backtest()
portfolio.rebalance()
portfolio.performance(show_all_rets=True)

2025-01-18 01:05:30,880 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Generating position for strategy MVO - RollingMean(60)|1 between 2023-02-06 and 2025-01-21......
100%|██████████| 512/512 [00:04<00:00, 109.97it/s]
2025-01-18 01:05:35,565 - TradingSubSystemSingle [MVO - RollingMean(60)|1] - INFO - Volatility Target = nan% | Price Volatility = 37.8% | Last Scale Factor = 1.00
2025-01-18 01:05:35,569 - TradingSubSystemSingle [MVO - RollingMean(30)|1] - INFO - Generating position for strategy MVO - RollingMean(30)|1 between 2023-02-06 and 2025-01-21......
100%|██████████| 512/512 [00:04<00:00, 114.32it/s]
2025-01-18 01:05:40,069 - TradingSubSystemSingle [MVO - RollingMean(30)|1] - INFO - Volatility Target = nan% | Price Volatility = 33.4% | Last Scale Factor = 1.00
2025-01-18 01:05:40,075 - TradingSubSystemSingle [MVO - RollingMean(10)|0.5] - INFO - Generating position for strategy MVO - RollingMean(10)|0.5 between 2023-02-06 and 2025-01-21......
100%|██████████| 512/512 

,Rebalanced Portfolio,Optimized Portfolio,^SPX,Equal Weighted,MVO - RollingMean(60)|1,MVO - RollingMean(30)|1,MVO - RollingMean(10)|0.5,MVO - RollingMean(5)|0.5,"MVO - RSI(10,2)|0.2",MVO - Double(5)|0.2
Measure,,,,,,,,,,
Cumulative Return,2.486850,2.331269,1.442546,2.117929,2.776532,2.350306,1.636117,1.384307,1.052677,1.317203
Annualized Return,0.498613,0.464951,0.196912,0.403895,0.558075,0.468939,0.262618,0.175960,0.028654,0.145202
Annualized Volatility,0.239949,0.238537,0.126567,0.183824,0.250583,0.237660,0.132666,0.128968,0.066413,0.080099
Annualized Sharpe Ratio,1.959694,1.830170,1.331514,2.042761,2.113827,1.853710,1.765570,1.144271,0.004031,1.458384
Maximum Drawdown,-0.150277,-0.153500,-0.102766,-0.121574,-0.178395,-0.161327,-0.111620,-0.081943,-0.104537,-0.040354


In [7]:
portfolio.portfolio_breakdown()
portfolio.instrument_breakdown()

In [8]:
portfolio.get_position_for_trade().tail(20)

2025-01-18 01:04:35,255 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $231,967
2025-01-18 01:04:35,256 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $231,967


,AAPL,BRK-B,DXJ,META,NANC,NVDA,QQQ,SPY,TSLA,CASH
Date,,,,,,,,,,
2024-12-18,80.0,69.0,115.0,43.0,258.0,75.0,26.0,14.0,222.0,4879.0
2024-12-19,80.0,69.0,115.0,43.0,258.0,75.0,26.0,14.0,222.0,4879.0
2024-12-20,131.0,41.0,125.0,18.0,186.0,150.0,18.0,12.0,159.0,44449.0
2024-12-23,131.0,41.0,125.0,18.0,186.0,150.0,18.0,12.0,159.0,44449.0
2024-12-24,131.0,41.0,125.0,18.0,186.0,150.0,18.0,12.0,159.0,44449.0
2024-12-26,131.0,41.0,125.0,18.0,186.0,150.0,18.0,12.0,159.0,44449.0
2024-12-27,214.0,11.0,195.0,13.0,216.0,281.0,22.0,14.0,133.0,13910.0
2024-12-30,214.0,11.0,195.0,13.0,216.0,281.0,22.0,14.0,133.0,13910.0
2024-12-31,214.0,11.0,195.0,13.0,216.0,281.0,22.0,14.0,133.0,13910.0


# Execute the Portfolio

In [9]:
executor = ExecutorFutu(is_test=False)
executor.set_portfolio(portfolio)
orders = executor.execute(px_interval='5m')
orders

2025-01-18 01:04:40,338 - ExecutorFutu - INFO - market: US
2025-01-18 01:04:40,339 - ExecutorFutu - INFO - Cancel all orders first before executing.....


2025-01-18 01:04:40,346 | 49194 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=1, host=127.0.0.1, port=11111, user_id=18214795
2025-01-18 01:04:40,508 | 49194 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=1


2025-01-18 01:04:43,513 - Standard Portfolio ({self.name}) - CRITICAL - Portfolio target capital is not specified, use initial backtest capital of $231,967
2025-01-18 01:04:43,513 - Standard Portfolio ({self.name}) - INFO - Generate trade position based on target capital of $231,967
2025-01-18 01:04:43,516 - ExecutorFutu - INFO - Execute Standard Portfolio ({self.name}) position based on 2025-01-17


2025-01-18 01:04:43,518 | 49194 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=2, host=127.0.0.1, port=11111, user_id=18214795


2025-01-18 01:04:43,530 - Futu - INFO - 9 Positions: US.TSLA, US.SPY, US.QQQ, US.NVDA, US.NANC, US.META, US.DXJ, US.BRK.B, US.AAPL


...

2025-01-18 01:05:06,944 - Futu - INFO - Placed Order: {'code': 'US.SPY', 'price': 597.82, 'qty': 9.0, 'trd_side': 'BUY', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2025-01-18 01:05:06,945 | 49194 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=10
2025-01-18 01:05:09,952 | 49194 | [open_context_base.py] _send_init_connect_sync:311: InitConnect ok: conn_id=11, host=127.0.0.1, port=11111, user_id=18214795


2025-01-18 01:05:10,202 - Futu - INFO - Placed Order: {'code': 'US.TSLA', 'price': 431.5, 'qty': 28.0, 'trd_side': 'BUY', 'order_type': 'NORMAL', 'market': 'US', 'trd_env': 'REAL'}


2025-01-18 01:05:10,203 | 49194 | [open_context_base.py] on_disconnect:383: Disconnected: conn_id=11


,code,stock_name,trd_side,order_type,order_status,order_id,qty,price,create_time,updated_time,...,remark,time_in_force,fill_outside_rth,aux_price,trail_type,trail_value,trail_spread,currency,portfolio,date
0,US.AAPL,苹果,SELL,NORMAL,SUBMITTING,872432321464365497,60.0,229.66,2025-01-17 12:04:44,2025-01-17 12:04:44,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17
0,US.BRK.B,伯克希尔-B,BUY,NORMAL,SUBMITTING,4784114847442415127,63.0,468.66,2025-01-17 12:04:47,2025-01-17 12:04:47,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17
0,US.DXJ,日本对冲股票ETF-WisdomTree,SELL,NORMAL,SUBMITTING,2432158181382279076,93.0,107.80,2025-01-17 12:04:50,2025-01-17 12:04:50,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17
0,US.META,Meta Platforms,SELL,NORMAL,SUBMITTING,333888972312135246,31.0,612.56,2025-01-17 12:04:54,2025-01-17 12:04:54,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17
0,US.NANC,UNUSUAL WHALES SUBVERSIVE DEMOCRATIC TRADING ETF,BUY,NORMAL,SUBMITTING,1165630772408004453,5.0,39.48,2025-01-17 12:04:57,2025-01-17 12:04:57,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17
0,US.NVDA,英伟达,SELL,NORMAL,SUBMITTING,7738854150840222617,10.0,138.02,2025-01-17 12:05:00,2025-01-17 12:05:00,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17
0,US.QQQ,纳指100ETF-Invesco QQQ Trust,BUY,NORMAL,SUBMITTING,313102323386604434,2.0,520.80,2025-01-17 12:05:03,2025-01-17 12:05:03,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17
0,US.SPY,标普500ETF-SPDR,BUY,NORMAL,SUBMITTING,5257491249769500616,9.0,597.82,2025-01-17 12:05:07,2025-01-17 12:05:07,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17
0,US.TSLA,特斯拉,BUY,NORMAL,SUBMITTING,7024455511803209904,28.0,431.50,2025-01-17 12:05:10,2025-01-17 12:05:10,...,Standard Portfolio ({self.name}),DAY,False,N/A,N/A,N/A,N/A,USD,Standard Portfolio ({self.name}),2025-01-17


# Calibration Portfolios